In [1]:
!pip install datasets evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00


In [2]:
# inferencing Base model
from transformers import AutoTokenizer,AutoModelForSequenceClassification
model_id='bert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(model_id)
model=AutoModelForSequenceClassification.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
test_data='I like this movie so much.'
from transformers import pipeline
pipe=pipeline('text-classification',tokenizer=tokenizer,model=model)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.6068554520606995}]

In [7]:
output=pipe(test_data)
output

[{'label': 'LABEL_0', 'score': 0.6068554520606995}]

In [9]:
if output[0]['label']=='LABEL_0':
  print("The result is negative.")
else:
  print("The result is positive")

The result is negative.


In [13]:
# Pytorch inference
tokenized_input=tokenizer(test_data,padding=True,return_tensors='pt')  #tf  #np
tokenized_input

{'input_ids': tensor([[ 101, 1045, 2066, 2023, 3185, 2061, 2172, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [17]:
import torch
model.eval()
with torch.no_grad():
  output=model(**tokenized_input)
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2625, -0.1716]]), hidden_states=None, attentions=None)

In [18]:
output=output.logits
output=torch.nn.functional.softmax(output)
output

<ipython-input-18-9d9429548569>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output=torch.nn.functional.softmax(output)


tensor([[0.6069, 0.3931]])

In [20]:
ans=torch.argmax(output)
ans

tensor(0)

In [21]:
class_label=['neg','pos']
final_output=class_label[ans]
final_output

'neg'

In [22]:
test_data=['I love this movie.',
           "I am not sure I like it.",
           "I do not like black color, but I always chose it whenever I go outside."]

In [26]:
token_test_data= tokenizer(test_data,truncation=True,padding=True,return_tensors='pt')
token_test_data

{'input_ids': tensor([[ 101, 1045, 2293, 2023, 3185, 1012,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [ 101, 1045, 2572, 2025, 2469, 1045, 2066, 2009, 1012,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [ 101, 1045, 2079, 2025, 2066, 2304, 3609, 1010, 2021, 1045, 2467, 4900,
         2009, 7188, 1045, 2175, 2648, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [28]:
import torch
model.eval()
with torch.no_grad():
  output=model(**token_test_data).logits
output


tensor([[ 0.2863, -0.1838],
        [ 0.3398, -0.1196],
        [ 0.2345, -0.1212]])

In [29]:
prob_value_output=torch.nn.functional.softmax(output,dim=-1)
prob_value_output

tensor([[0.6154, 0.3846],
        [0.6129, 0.3871],
        [0.5880, 0.4120]])

In [30]:
outputs=torch.argmax(prob_value_output,dim=-1)
outputs

tensor([0, 0, 0])

# Inferencing with Tensorflow

In [31]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
model=TFAutoModelForSequenceClassification.from_pretrained(model_id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
data="I like this color."
token_data=tokenizer(data,truncation=True,return_tensors='tf')
output=model(token_data)
output

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.8062439, -0.5817733]], dtype=float32)>, hidden_states=None, attentions=None)

In [36]:
output=output.logits
prob_value=tf.nn.softmax(output)
prob_value

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.8002755, 0.1997245]], dtype=float32)>

In [42]:
final_output=tf.argmax(prob_value).numpy()[0]

In [43]:
class_label[final_output]

'neg'

# Fine tuning

In [44]:
from datasets import load_dataset
ds=load_dataset('rotten_tomatoes')
ds

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [47]:
ds['train'][0]['text']

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

In [51]:
len(ds['train'][1000]['text'])

127

In [57]:
def compute_length(data):
  return {'length':len(data['text'])}

ds['train']=ds['train'].map(compute_length)
ds['train']

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'length'],
    num_rows: 8530
})

In [58]:
ds['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1,
 'length': 177}

In [59]:
max(ds['train']['length'])

267

In [45]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [75]:
def token_function(data):
  return tokenizer(data['text'],truncation=True,padding='max_length',max_length=270,return_tensors='pt')

train_token=ds['train'].map(token_function,batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

In [63]:
train_token

Dataset({
    features: ['text', 'label', 'length', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8530
})



---



In [77]:
valid_token=ds['validation'].map(token_function,batched=True)
test_token=ds['test'].map(token_function,batched=True)

In [78]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [66]:
import evaluate
accuracy = evaluate.load("accuracy")

In [79]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [80]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [69]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
training_args = TrainingArguments(
    output_dir="./finetune",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_token,
    eval_dataset=valid_token,
    tokenizer=tokenizer,
    #data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.399300,0.348373,0.860225
2,0.222100,0.404700,0.863977


TrainOutput(global_step=1068, training_loss=0.30407842803983653, metrics={'train_runtime': 858.927, 'train_samples_per_second': 19.862, 'train_steps_per_second': 1.243, 'total_flos': 2367074498436000.0, 'train_loss': 0.30407842803983653, 'epoch': 2.0})

In [ ]:
pipeline('text-classification',model='nnhwin/finetune')